In [11]:
import numpy as np
import sqlite3
import plotly.express as px
import pandas as pd
import sqlite3
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import statsmodels
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA
from useful_functions_2 import *
from pdf2image import convert_from_path
from sklearn.manifold import TSNE
from scipy.spatial.distance import braycurtis
from skbio.stats.distance import permanova
from scipy.spatial.distance import pdist
from skbio.stats.distance import DistanceMatrix

### Leaf Microbiome Dataframe

In [2]:
tables_to_join = ["tPlantMicrobeGenus", "tSample", "tPlantDNA"]
common_column = "sample_id"
full_df = import_all_data(tables_to_join, common_column)
print(full_df.shape)
full_df = full_df.T.drop_duplicates().T
subdf = full_df.copy()
origin_pivoted_df = subdf.pivot(index=['sample_id', 'site_id', 'species', 'V2'], columns='genera', values='relative_abundance')
origin_pivoted_df.shape
origin_pivoted_df.columns.name = None
origin_pivoted_df = origin_pivoted_df.reset_index()
print(origin_pivoted_df.shape)
origin_pivoted_df.head()

(43811, 15)


,sample_id,site_id,species,V2,Archaea|Euryarchaeota|Halobacteria|Halobacteriales|Haloarculaceae|Halapricum,Archaea|Euryarchaeota|Halobacteria|Halobacteriales|Halobacteriaceae|Halorussus,Archaea|Euryarchaeota|Halobacteria|Halobacteriales|Halococcaceae|Halococcus,Archaea|Euryarchaeota|Halobacteria|Haloferacales|Haloferacaceae|Halobellus,Archaea|Euryarchaeota|Halobacteria|Haloferacales|Haloferacaceae|Haloferax,Archaea|Euryarchaeota|Halobacteria|Haloferacales|Haloferacaceae|Halogeometricum,...,Eukaryota|Basidiomycota|Ustilaginomycetes|Ustilaginales|Ustilaginaceae|Ustilago,Eukaryota|Basidiomycota|Wallemiomycetes|Wallemiales|Wallemiaceae|Wallemia,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Bathycoccaceae|Bathycoccus,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Bathycoccaceae|Ostreococcus,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Mamiellaceae|Micromonas,Eukaryota|Chlorophyta|Trebouxiophyceae|Chlorellales|Chlorellaceae|Chlorella,Eukaryota|Chlorophyta|Trebouxiophyceae|Chlorellales|Chlorellaceae|Prototheca,Eukaryota|Mucoromycota|Mortierellomycetes|Mortierellales|Mortierellaceae|Mortierella,Eukaryota|Mucoromycota|Mucoromycetes|Mucorales|Rhizopodaceae|Rhizopus,Eukaryota|Mucoromycota|Umbelopsidomycetes|Umbelopsidales|Umbelopsidaceae|Umbelopsis
0,CMB10,CMB,syriaca,0.999777,0.03005,NaN,0.016,NaN,NaN,NaN,...,0.00013,NaN,NaN,NaN,NaN,NaN,0.00014,0.00011,NaN,NaN
1,CMB11,CMB,syriaca,1.000000,0.02488,NaN,0.00903,NaN,NaN,NaN,...,0.00021,NaN,0.00011,NaN,0.00011,NaN,0.00016,NaN,NaN,NaN
2,CMB13,CMB,syriaca,0.989131,0.02391,NaN,0.01344,NaN,NaN,NaN,...,0.00013,NaN,0.00026,NaN,0.00032,NaN,0.00017,0.00016,NaN,NaN
3,CMB14,CMB,syriaca,0.999992,0.02786,NaN,0.00968,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00018,NaN,0.00021,NaN,NaN,NaN
4,CMB15,CMB,syriaca,0.998548,0.00774,NaN,0.00315,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
origin_pivoted_df.to_csv('full_leaf_microbiome.csv')

### Soil Microbiome Dataframe

In [4]:
taxa_level = 'Genera'
table_name = 'tSoilMicrobe' + taxa_level
tables_to_join = [table_name, "tSample", "tPlantDNA"]
common_column = "sample_id"
full_df = import_all_data(tables_to_join, common_column)
full_df = full_df.T.drop_duplicates().T
full_df.shape
full_df.rename(columns={'genera': 'taxa'}, inplace=True)
opdf = full_df.pivot(index=['sample_id', 'site_id', 'species', 'V2'], columns='taxa', values='relative_abundance')
opdf = opdf.reset_index()
opdf = opdf.fillna(0)    
col_list1 = []
for i in opdf.columns:
    col_list1.append(i.split('|')[-1])
opdf.columns = col_list1
taxa_columns = opdf.columns
taxa_columns = taxa_columns.drop(['sample_id','site_id', 'species', 'V2'])

In [5]:
site_list = opdf['site_id']

opdf['BGR'] = site_list

opdf['BGR'].replace({'CMB': 1, 'FRW': 1, 'LFS': 1, 'LM': 0, 'MMP': 1, 'MKP': 2,
                    'PNR': 2, 'PTW': 0, 'RF': 2, 'RGT': 0, 'RRL': 0, 'SGC': 0, 'SLG': 1,'HR': 3}, inplace=True) #
opdf = opdf[opdf["BGR"].isin([0,1,2,3])]

opdf['BGR'] = opdf['BGR'].replace([0], 'Wintergreen')
opdf['BGR'] = opdf['BGR'].replace([1], 'Cole Mountain')
opdf['BGR'] = opdf['BGR'].replace([2], 'Blacksburg')
opdf['BGR'] = opdf['BGR'].replace([3], 'HR')

C:\Users\15404\AppData\Local\Temp\ipykernel_6456\921536217.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  opdf['BGR'] = site_list


In [6]:
### Reduce the data by removing Sequence Run 1
libdf = pd.read_csv('lib_samples.csv')

sample_list = opdf['sample_id']
opdf['seq_id'] = sample_list

for sample1 in libdf['lib_one']:
    opdf['seq_id'] = opdf['seq_id'].replace([sample1], '1')
for sample2 in libdf['lib_two']:
    opdf['seq_id'] = opdf['seq_id'].replace([sample2], '2')
    
sample_list = opdf['seq_id'].unique().tolist()
sample_list.remove('1')
#sample_list.remove('2')
opdf['seq_id'].replace(sample_list, "3", inplace=True)

opdf = opdf[opdf['seq_id'].isin(['3'])]
opdf = opdf[opdf["species"].isin(['syriaca','exaltata', 'hybrid'])]

In [7]:
print(opdf.shape)
opdf.head()

(87, 2532)


,sample_id,site_id,species,V2,Candidatus Korarchaeum,Acidilobus,Caldisphaera,Aeropyrum,Desulfurococcus,Ignicoccus,...,Iridovirus,Phix174microvirus,Hokovirus,Klosneuvirus,Tupanvirus,Alphapartitivirus,Bracovirus,Pandoravirus,BGR,seq_id
0,CMB10,CMB,syriaca,0.999777,0.0,0.0,0.0,0.00068,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,Cole Mountain,3
1,CMB11,CMB,syriaca,1.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,Cole Mountain,3
2,CMB13,CMB,syriaca,0.989131,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,Cole Mountain,3
3,CMB15,CMB,syriaca,0.998548,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,Cole Mountain,3
4,CMB16,CMB,syriaca,0.999243,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.00308,0.0,0.0,0.0,0.0,Cole Mountain,3


In [8]:
opdf.to_csv('full_soil_microbiome.csv')

### Soil Microbiome Dataframe with Soil Nutrients

In [12]:
taxa_level = 'Genera'
table_name = 'tSoilMicrobe' + taxa_level
tables_to_join = ['tSample', 'tSoilNutrients', 'tPlantDNA', table_name]
common_column = "sample_id"
full_df = import_all_data(tables_to_join, common_column)
print(full_df.shape)
full_df = full_df.T.drop_duplicates().T
print(full_df.shape)
full_df.rename(columns={'genera': 'taxa'}, inplace=True)
opdf = full_df.pivot(index=['sample_id', 'site_id', 'species', 'V2', "Phosphorus", "Potassium", "Calcium", "Magnesium", "Sulfur", "Sodium", "Zinc", "Manganese", "Iron", "Copper", "Boron"], columns='taxa', values='relative_abundance')

(383952, 32)
(383952, 29)


In [13]:
opdf = opdf.reset_index()
opdf.columns.name = None
col_list1 = []
for i in opdf.columns:
    col_list1.append(i.split('|')[-1])
opdf.columns = col_list1
opdf.head()

,sample_id,site_id,species,V2,Phosphorus,Potassium,Calcium,Magnesium,Sulfur,Sodium,...,Begomovirus,Inovirus,Iridovirus,Phix174microvirus,Hokovirus,Klosneuvirus,Tupanvirus,Alphapartitivirus,Bracovirus,Pandoravirus
0,CMB10,CMB,syriaca,0.999777,79.0,98.0,620.0,104.0,27.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CMB11,CMB,syriaca,1.000000,105.0,95.0,649.0,115.0,25.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CMB13,CMB,syriaca,0.989131,60.0,91.0,805.0,139.0,20.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CMB15,CMB,syriaca,0.998548,36.0,122.0,713.0,139.0,22.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CMB16,CMB,syriaca,0.999243,51.0,155.0,1222.0,208.0,22.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.00308,0.0,0.0,0.0,0.0


In [14]:
### Reduce the data by removing Sequence Run 1
libdf = pd.read_csv('lib_samples.csv')

sample_list = opdf['sample_id']
opdf['seq_id'] = sample_list

for sample1 in libdf['lib_one']:
    opdf['seq_id'] = opdf['seq_id'].replace([sample1], '1')
for sample2 in libdf['lib_two']:
    opdf['seq_id'] = opdf['seq_id'].replace([sample2], '2')
    
sample_list = opdf['seq_id'].unique().tolist()
sample_list.remove('1')
#sample_list.remove('2')
opdf['seq_id'].replace(sample_list, "3", inplace=True)

opdf = opdf[opdf['seq_id'].isin(['3'])]
opdf = opdf[opdf["species"].isin(['syriaca','exaltata'])]

In [15]:
print(opdf.shape)

(83, 2542)


In [16]:
opdf.head()

,sample_id,site_id,species,V2,Phosphorus,Potassium,Calcium,Magnesium,Sulfur,Sodium,...,Inovirus,Iridovirus,Phix174microvirus,Hokovirus,Klosneuvirus,Tupanvirus,Alphapartitivirus,Bracovirus,Pandoravirus,seq_id
0,CMB10,CMB,syriaca,0.999777,79.0,98.0,620.0,104.0,27.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,CMB11,CMB,syriaca,1.000000,105.0,95.0,649.0,115.0,25.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,CMB13,CMB,syriaca,0.989131,60.0,91.0,805.0,139.0,20.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,CMB15,CMB,syriaca,0.998548,36.0,122.0,713.0,139.0,22.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,CMB16,CMB,syriaca,0.999243,51.0,155.0,1222.0,208.0,22.0,16.0,...,0.0,0.0,0.0,0.0,0.00308,0.0,0.0,0.0,0.0,3


In [17]:
opdf.to_csv('micro_nutrients.csv')

### Leaf Microbiome Dataframe with Leaf Nutrients

In [8]:
tables_to_join = ["tPlantMicrobeGenus", "tSample", "tPlantDNA", 'tPlantNutrients']
common_column = "sample_id"
full_df = import_all_data(tables_to_join, common_column)
full_df = full_df.T.drop_duplicates().T
subdf = full_df.copy()
origin_pivoted_df = subdf.pivot(index=['sample_id', 'site_id', 'species', 'V2', 'N_percent_dm', 'P_percent_dm',
       'K_percent_dm', 'Ca_percent_dm', 'Mg_percent_dm', 'Na_percent_dm',
       'S_percent_dm', 'Zn_ppm_dm', 'Mn_ppm_dm', 'Fe_ppm_dm', 'Cu_ppm_dm',
       'B_ppm_dm', 'Al_ppm_dm'], columns='genera', values='relative_abundance')
origin_pivoted_df.shape
origin_pivoted_df.columns.name = None
origin_pivoted_df = origin_pivoted_df.reset_index()
print(origin_pivoted_df.shape)
origin_pivoted_df.head()

(158, 1443)


,sample_id,site_id,species,V2,N_percent_dm,P_percent_dm,K_percent_dm,Ca_percent_dm,Mg_percent_dm,Na_percent_dm,...,Eukaryota|Basidiomycota|Ustilaginomycetes|Ustilaginales|Ustilaginaceae|Ustilago,Eukaryota|Basidiomycota|Wallemiomycetes|Wallemiales|Wallemiaceae|Wallemia,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Bathycoccaceae|Bathycoccus,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Bathycoccaceae|Ostreococcus,Eukaryota|Chlorophyta|Mamiellophyceae|Mamiellales|Mamiellaceae|Micromonas,Eukaryota|Chlorophyta|Trebouxiophyceae|Chlorellales|Chlorellaceae|Chlorella,Eukaryota|Chlorophyta|Trebouxiophyceae|Chlorellales|Chlorellaceae|Prototheca,Eukaryota|Mucoromycota|Mortierellomycetes|Mortierellales|Mortierellaceae|Mortierella,Eukaryota|Mucoromycota|Mucoromycetes|Mucorales|Rhizopodaceae|Rhizopus,Eukaryota|Mucoromycota|Umbelopsidomycetes|Umbelopsidales|Umbelopsidaceae|Umbelopsis
0,CMB10,CMB,syriaca,0.999777,3.53,0.43,2.03,1.17,0.56,0.01,...,0.00013,NaN,NaN,NaN,NaN,NaN,0.00014,0.00011,NaN,NaN
1,CMB11,CMB,syriaca,1.000000,3.94,0.44,1.86,0.84,0.48,0.01,...,0.00021,NaN,0.00011,NaN,0.00011,NaN,0.00016,NaN,NaN,NaN
2,CMB13,CMB,syriaca,0.989131,3.38,0.39,2.54,0.84,0.44,0.01,...,0.00013,NaN,0.00026,NaN,0.00032,NaN,0.00017,0.00016,NaN,NaN
3,CMB14,CMB,syriaca,0.999992,3.10,0.34,2.47,0.75,0.34,0.01,...,NaN,NaN,NaN,NaN,0.00018,NaN,0.00021,NaN,NaN,NaN
4,CMB15,CMB,syriaca,0.998548,3.43,0.30,2.21,0.64,0.25,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
origin_pivoted_df.to_csv('leaf_micro_nutrients.csv')